In [2]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
from datetime import datetime
from io import BytesIO
from PIL import Image
import boto3
from selenium.webdriver.common.by import By



In [3]:

# 페이지 크롤링 함수
def cardCrawling (cardurl):
    benefits=""
    
    cardhtml=urlopen(cardurl)
    cardbs=BeautifulSoup(cardhtml,'html.parser')
    
    # benefits
    benes = cardbs.findAll('div',{'class','bnfCont'})
    if len(benes) == 0:
        beneLists = cardbs.find('ul',{'class','toggleList'}).findAll('li',recursive=False)
        for beneList in beneLists:
            titlebene = beneList.find('a').text.replace('\t','').replace('\r','').replace('\n','').replace(' ','')
            if titlebene=="L.POINT" or titlebene=="가족카드" or titlebene== "가족카드안내" or titlebene== "연회비": continue
            benefits+="<"+titlebene+"> "
            details = beneList.find('div',{'class','toggleCont'}).findAll(recursive=False)
            for detail in details:
                if str(detail)[1:6]=="table": 
                    benefits+=str(detail).replace('\n','')
                elif str(detail)[1:3]=="h3": 
                    benefits+="["+detail.text+"]"
                elif str(detail)[1:3]=="h4": 
                    benefits+="/"+detail.text+": "
                elif str(detail)[1:6]=="style": 
                    continue
                else:
                    if detail.find('table') is not None:
                        benefits+=str(detail).replace('\n','')
                    else:
                        benefits+=detail.text.replace('\n','')
                benefits.replace('\n','').replace('\t','')
            benefits+="\n"
    else:
        for bene in benes:
            titlebene=bene.find('h3').text
            if titlebene=="L.POINT" or titlebene=="가족카드" or titlebene== "가족카드 안내" or titlebene== "연회비" or titlebene== "혜택 모아보기": continue
            benefits+="<"+titlebene+">"
            sections = bene.findAll('div',{'class','toggle'})
            for section in sections:
                beneNames=section.find('h4')
                beneNameText=beneNames.text
                benefits+="["+beneNameText+"] "
                details = section.find('div',{'class','toggleCont'}).findAll(recursive=False)

                for detail in details:
                    if str(detail)[1:6]=="table": 
                        benefits+=str(detail).replace('\n','')
                    elif str(detail)[1:3]=="h3": 
                        benefits+="["+detail.text+"]"
                    elif str(detail)[1:3]=="h4": 
                        benefits+="/"+detail.text+": "
                    elif str(detail)[1:6]=="style": 
                        continue
                    else:
                        if detail.find('table') is not None:
                            benefits+=str(detail).replace('\n','')
                        else:
                            benefits+=detail.text.replace('\n','')
                benefits.replace('\n','').replace('\r','').replace('\t','')
            benefits+="\n"
            benefits=benefits.replace("'","")
            
    return benefits 


In [4]:

def cardList(associate):
    url = 'https://www.lottecard.co.kr/app/LPCDADA_V100.lc'

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)

    # 웹 페이지 로드
    driver.get(url)

    if(associate):
        # 제휴 카드 클릭
        link = driver.find_element(By.LINK_TEXT,"제휴")
        link.click()
        time.sleep(5)

    # Selenium으로 페이지 스크랩
    rendered_html = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(rendered_html, 'html.parser')

    # 더보기 버튼 끝까지 누르기
    while True:
        new_render_html=driver.page_source
        soup = BeautifulSoup(new_render_html, 'html.parser')
        if soup.find('button',{'id':'btnMore'}) is None: break
        driver.find_element("id","btnMore").click()
        time.sleep(3)

    return soup.find('ul', {'id':'ajaxCardList'}).findAll('li')



In [5]:

name = []
img_url = []
benefits = []
created_at = []


In [6]:

cardlist=cardList(False)

In [7]:

for card in cardlist:
    cardNo = card.find('a').get('onclick')
    cardNo = re.search(r"'(.*?)'", cardNo).group(1)
    
    cardurl='https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC='+cardNo
    
    cardImg= "https:" + card.find('img').get('src')
    img_url.append("https://once-s3.s3.ap-northeast-2.amazonaws.com/lottecard/"+cardNo+".png")
    
    cardName=card.find('b').text
    name.append(cardName)

    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S")+" ["+cardName+"] --- 웹 페이지에 접속 중... ")
    
    benefit = cardCrawling(cardurl)
    benefits.append(benefit)

    now_datetime = datetime.now()
    formatted_now = now_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")
    created_at.append(formatted_now)

2024-04-09 07:29:22 [롯데마트&MAXX 카드] --- 웹 페이지에 접속 중... 


2024-04-09 07:29:22 [디지로카 London] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:23 [디지로카 Paris] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:23 [LOCA LIKIT 1.2] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:23 [LOCA in MEGACITY] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:24 [LOCA LIKIT Eat] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:24 [롤라카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:24 [LOCA LIKIT Play] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:25 [LOCA for Shopping] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:25 [LOCA LIKIT Shop] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:25 [LOCA 환승 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:25 [LOCA LIKIT] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:26 [LOCA 나누기 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:26 [LOCA 100 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:26 [LOCA CLASSIC] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:27 [ LOCA for Coffee ] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:27 [LOCA for Auto] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:27 [LOCA for Edu] --- 웹 페이지에 접속 중... 
2024-04-09 07:29:28 [LOCA for Health] --- 웹 페이지에 접속 중... 

In [8]:
cardlist2=cardList(True)

In [9]:
for card in cardlist2:
    cardNo = card.find('a').get('onclick')
    cardNo = re.search(r"'(.*?)'", cardNo).group(1)
    
    cardurl='https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC='+cardNo
    
    cardImg= "https:" + card.find('img').get('src')
    img_url.append("https://once-s3.s3.ap-northeast-2.amazonaws.com/lottecard/"+cardNo+".png")
    
    cardName=card.find('b').text
    name.append(cardName)

    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S")+" ["+cardName+"](제휴) --- 웹 페이지에 접속 중... ")
    
    benefit = cardCrawling(cardurl)
    benefits.append(benefit)

    now_datetime = datetime.now()
    formatted_now = now_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")
    created_at.append(formatted_now)

2024-04-09 07:30:26 [Trip to 로카](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:26 [카카오뱅크 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:29 [케이뱅크 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:29 [Trip to 로카](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:30 [Kakaopage 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:30 [LOTTE ONers 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:30 [LOCA X 웰릭스렌탈 ](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:31 [L.pay 롯데카드 Ⅱ](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:31 [교보문고 플래티넘 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:31 [롯데백화점 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:31 [롯데백화점 리빙 by 롤라 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:32 [OIL KING SK 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:32 [KT Super DC 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:32 [경차 smart 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:33 [The CJ 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:33 [AXA다이렉트 프라임 롯데카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:30:33 [메가 멤버스 롯데카드](제휴) -

In [10]:
    
card_company_id = [5] * len(name)
type = ["CreditCard"] * len(name)

data = {"card_company_id":card_company_id, "name" : name, "img_url" : img_url, "benefits": benefits, "created_at": created_at,"type":type}
df = pd.DataFrame(data)
 
df.to_csv("./credit_benefit.csv", encoding = "utf-8-sig", index=False)
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S")+" 롯데카드 신용카드 크롤링 완료")


2024-04-09 07:31:07 롯데카드 신용카드 크롤링 완료
